# Golden SAML AD FS Mail Access Hunt

### Scope the Environment
- Identity Provider (IdP) -  On-Prem Active Directory Federation Services (AD FS) server w/ Azure AD Connect
- Cloud Service Provider (SP) - Microsoft 365 (Mail, Defender)
- On-Prem Workstations

### Dataset
https://securitydatasets.com/notebooks/compound/GoldenSAMLADFSMailAccess.html
- WindowsEvents.Zip
- Microsoft365DefenderEvents.Zip
- AADAuditEvents.Zip
- OfficeActivityEvents.Zip

#### Simple Data Ingestion

All data is unzipped and loaded to two SQlite stores (SQL-supported lakehouse simulation) using [kestrel-tool::mkdb](https://github.com/opencybersecurityalliance/kestrel-lang/blob/develop/packages/kestrel_tool/src/kestrel_tool/mkdb.py)

#### Simulated On-premise Datastore `sqlite:///onpremise.db`

| Data File | Datastore Index | Kestrel Datasource | ........................................................................................................................................................................ |
| :------- | :------ | :------ | ------- |
| WindowsEvents.Zip | windows | GoldenSAML-WindowsEvents ||

#### Simulated Cloud Datastore `sqlite:///cloud.db`

| Data File | Datastore Index | Kestrel Datasource | .............................................................................................................................. |
| :------- | :------ | :------ | ------- |
| Microsoft365DefenderEvents.Zip | msdefender | GoldenSAML-Microsoft365DefenderEvents||
| AADAuditEvents.Zip | aad | GoldenSAML-AADAuditEvents ||
| OfficeActivityEvents.Zip | office | GoldenSAML-OfficeActivityEvents||

### Campaign Hypothesis
Cyber Threat Actor (CTA) will compromise the organization's identity provider (AD FS) and attempt to obtain the private key used for signing SAML tokens, to then forge a valid SAML token that will impersonate a legitimate user to access sensitive data hosted by the organization's cloud service provider (Microsoft 365). 

#### Mitre Att&ck Mapping


## Steps in This Huntbook

1. Read the [GoldenSAML blog](https://www.cyberark.com/resources/threat-research-blog/golden-saml-newly-discovered-attack-technique-forges-authentication-to-cloud-apps) to understand the generic attack flow
2. Develop the threat hypothesis around the [SimuLand GoldenSAML attack simulation](https://simulandlabs.com/labs/GoldenSAML/README.html)
3. Using the blog, attack simulation, and Mitre mapping above, develop hunt analytics around:
    - [T1003 - OS Credential Dumping](https://attack.mitre.org/techniques/T1003/)
    - [T1087.002 - Account Discovery: Domain Account](https://attack.mitre.org/techniques/T1087/002/)
    - [T1098.002: Additional Email Delegate Permissions](https://attack.mitre.org/techniques/T1098/002/)
    - [T1134: Access Token Manipulation](https://attack.mitre.org/techniques/T1134/)
    - [T1552.004 - Unsecured Credentials: Private Keys](https://attack.mitre.org/techniques/T1552/004/)
4. Move across multiple data sources to verify different phases of the attack from multiple angles (already written in this huntbook)
5. **Execute this huntbook and report your findings**
6. Further drill down
    - Add new cells in this huntbook to explore other paths of the hunt
    - Explore other aspects of the attack with any *[Quiz]* huntbook in the same folder
    - Check the *Kestrel Execution Explained* huntbook to get a basic idea of the Kestrel abstraction

In [1]:
# Let's first understand what data we have available in our datasets
# `type_uid` is a required attribute of OCSF event/activity
# type_uid LIKE '%' means any event

win_events = GET event FROM sqlalchemy://GoldenSAML-WindowsEvents
             WHERE type_uid LIKE '%'
             START 2021-08-02T00:00:00.000Z STOP 2021-08-03T00:00:00.000Z

# We use customized OCSF field `event_id` for holding raw event id (if exist in source)
# We use OCSF field `type_name` for holding event name

DISP win_events ATTR event_id, type_name

event_id,type_name
412,412
501,Desktop Window Manager is experiencing heavy resource contention
4624,An account was successfully logged on
33205,SQL audit event
4662,An operation was performed on an object
18,Pipe connected
5156,The Windows Filtering Platform has permitted a connection


In [2]:
mde_events = GET event FROM sqlalchemy://GoldenSAML-Microsoft365DefenderEvents
             WHERE type_uid LIKE '%'
             START 2021-08-02T00:00:00.000Z STOP 2021-08-03T00:00:00.000Z
 
DISP mde_events ATTR type_name

type_name
LdapSearch
Directory Services replication
MailItemsAccessed
Add delegated permission grant.


In [3]:
aad_events = GET event FROM sqlalchemy://GoldenSAML-AADAuditEvents
             WHERE type_uid LIKE '%'
             START 2021-08-02T00:00:00.000Z STOP 2021-08-03T00:00:00.000Z

DISP aad_events ATTR type_name

type_name
Update application – Certificates and secrets management
Update application
Add delegated permission grant


In [4]:
office_events = GET event FROM sqlalchemy://GoldenSAML-OfficeActivityEvents
                WHERE type_uid LIKE '%'
                START 2021-08-02T00:00:00.000Z STOP 2021-08-03T00:00:00.000Z

DISP office_events ATTR type_name

type_name
MailItemsAccessed


Now that we know what type of events we have available to use.  Let's see how these events apply to the attack
steps that were broken down in [Simulandlabs Golden SAML Overview](https://simulandlabs.com/labs/GoldenSAML/README.html).

The steps show that the actor would need to:
- Export the AD FS Configuration
- Export the AD FS DKM Master Key to decrypt the private key
- Export the AD FS Certificates
- Forge the SAML Tokens
- Get OAuth Access Token with SAML Assertion
- Finally, access mail messages using MS Graph


A few of these steps we may not be able to see because they can be conducted locally on the attacker's system.  Let's see what we can identify!!

Let's start by looking for any activity around the AD FS configuration.  Doing a quick Google search shows us that
[configuration](https://learn.microsoft.com/en-us/windows-server/identity/ad-fs/technical-reference/the-role-of-the-ad-fs-configuration-database) resides in the Windows Internal Database (WID) or a SQL Server.

What events can we see that might point us towards a database?

Windows Event ID [33205](https://www.ultimatewindowssecurity.com/sqlserver/auditlog/sampleevent.aspx) shows us SQL Server Audit events.

In [5]:
# Let's write a query to view all events from Event ID 33205

sql_events = GET event FROM sqlalchemy://GoldenSAML-WindowsEvents
             WHERE event_id = 33205
             
DISP sql_events

time,device.hostname,file.endpoint.hostname,process.endpoint.hostname,user.endpoint.hostname,actor.process.endpoint.hostname,device.uid,file.endpoint.uid,process.endpoint.uid,user.endpoint.uid,actor.process.endpoint.uid,file.type_id,type_uid,event_id,type_name
2021-08-02T13:05:33.483Z,adfs01.simulandlabs.com,adfs01.simulandlabs.com,adfs01.simulandlabs.com,adfs01.simulandlabs.com,adfs01.simulandlabs.com,/subscriptions/00000000-0000-0000-0000-000000000000/resourcegroups/azhybrid/providers/microsoft.compute/virtualmachines/adfs01,/subscriptions/00000000-0000-0000-0000-000000000000/resourcegroups/azhybrid/providers/microsoft.compute/virtualmachines/adfs01,/subscriptions/00000000-0000-0000-0000-000000000000/resourcegroups/azhybrid/providers/microsoft.compute/virtualmachines/adfs01,/subscriptions/00000000-0000-0000-0000-000000000000/resourcegroups/azhybrid/providers/microsoft.compute/virtualmachines/adfs01,/subscriptions/00000000-0000-0000-0000-000000000000/resourcegroups/azhybrid/providers/microsoft.compute/virtualmachines/adfs01,33205,600500,33205,SQL audit event


There is not alot of information from this event but we can see that some action happened on our `adfs01` host.

Let's see if there are other events around that host. 

In [6]:
procs = FIND process RESPONDED win_events
        WHERE device.hostname LIKE "%adfs01%"

DISP procs ATTR name, pid, endpoint.hostname

name,pid,endpoint.hostname
None,,adfs01.simulandlabs.com
powershell.exe,5692.0,adfs01.simulandlabs.com


There is a powershell process that's running on `adfs01`. Let's see what that process is doing.

In [7]:
powershell_event = FIND event RESPONDED BY procs

DISP powershell_event

time,device.hostname,file.endpoint.hostname,process.endpoint.hostname,user.endpoint.hostname,actor.process.endpoint.hostname,device.uid,file.endpoint.uid,process.endpoint.uid,user.endpoint.uid,actor.process.endpoint.uid,file.name,file.type_id,type_uid,event_id,type_name,process.name,process.file.name,process.file.path,process.file.parent_folder,actor.process.name,actor.process.file.name,actor.process.file.path,actor.process.file.parent_folder,process.pid,actor.process.pid,process.uid,actor.process.uid
2021-08-02T13:05:32.77Z,adfs01.simulandlabs.com,adfs01.simulandlabs.com,adfs01.simulandlabs.com,adfs01.simulandlabs.com,adfs01.simulandlabs.com,/subscriptions/00000000-0000-0000-0000-000000000000/resourcegroups/azhybrid/providers/microsoft.compute/virtualmachines/adfs01,/subscriptions/00000000-0000-0000-0000-000000000000/resourcegroups/azhybrid/providers/microsoft.compute/virtualmachines/adfs01,/subscriptions/00000000-0000-0000-0000-000000000000/resourcegroups/azhybrid/providers/microsoft.compute/virtualmachines/adfs01,/subscriptions/00000000-0000-0000-0000-000000000000/resourcegroups/azhybrid/providers/microsoft.compute/virtualmachines/adfs01,/subscriptions/00000000-0000-0000-0000-000000000000/resourcegroups/azhybrid/providers/microsoft.compute/virtualmachines/adfs01,\MICROSOFT##WID\tsql\query,6,100114,18,Pipe connected,powershell.exe,powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0,powershell.exe,powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0,5692,5692,9acedb82-ccd4-6107-22de-030000000600,9acedb82-ccd4-6107-22de-030000000600


This PowerShell process is running a query to the `WID` on `ADFS01`.  Someone is interacting with our WID database!
Let's see if defender caught any activity from this process id.

In [8]:
powershell_mde_events = GET event FROM sqlalchemy://GoldenSAML-Microsoft365DefenderEvents
                        WHERE actor.process.pid = procs.pid
                          AND device.hostname = procs.endpoint.hostname
DISP powershell_mde_events ATTR time, type_name, device.hostname SORT BY time ASC

mde_actor_procs = FIND process ORIGINATED mde_events
DISP mde_actor_procs

mde_actor_users = FIND user ORIGINATED mde_events
DISP mde_actor_users

time,type_name,device.hostname
2021-08-02T13:11:39.7724491Z,LdapSearch,adfs01.simulandlabs.com
2021-08-02T13:11:39.7747609Z,LdapSearch,adfs01.simulandlabs.com
2021-08-02T13:11:53.5836949Z,LdapSearch,adfs01.simulandlabs.com
2021-08-02T13:11:53.6344905Z,LdapSearch,adfs01.simulandlabs.com
2021-08-02T13:15:30.513652Z,LdapSearch,adfs01.simulandlabs.com


endpoint.hostname,parent_process.endpoint.hostname,endpoint.uid,parent_process.endpoint.uid,cmd_line,pid,uid,parent_process.pid,parent_process.uid,parent_process.file.name,file.name,file.path,file.parent_folder,file.hash.md5,file.hash.sha1,file.hash.sha256
adfs01.simulandlabs.com,adfs01.simulandlabs.com,00000000-0000-0000-0000-000000000000,00000000-0000-0000-0000-000000000000,powershell.exe,5692.0,5692.0,820.0,820.0,svchost.exe,powershell.exe,c:\windows\system32\windowspowershell\v1.0\powershell.exe,c:\windows\system32\windowspowershell\v1.0,7353f60b1739074eb17c5f4dddefe239,6cbce4a295c163791b60fc23d285e6d84f28ee4c,de96a6e69944335375dc1ac238336066889d9ffc7d73628ef4fe1b1b160ab32c
adfs01.simulandlabs.com,adfs01.simulandlabs.com,None,None,None,,,,,None,None,None,None,None,None,None
,,None,None,None,,,,,None,None,None,None,None,None,None


endpoint.hostname,endpoint.uid,uid,name,domain
adfs01.simulandlabs.com,00000000-0000-0000-0000-000000000000,S-1-5-21-2490359158-13971675-3780420524-500,wardog,simulandlabs
adfs01.simulandlabs.com,None,S-1-5-21-2490359158-13971675-3780420524-1105,pgustavo,simulandlabs.com
,None,,,


Using the PowerShell `PID`, we are able to see that there are also `ldapsearch` queries executing on `ADFS01` and who is executing those queries. What are those `ldapsearch` events doing?

In [9]:
ldap_query = FIND query_info RESPONDED BY powershell_mde_events

DISP ldap_query

uid,attr_list,search_filter
113597,"[""objectClass""]",(objectClass=*)
113598,"[""thumbnailphoto""]",(&(objectclass=contact)(!name=CryptoPolicy)(ThumbnailPhoto=*))
113608,"[""""]",(name=CryptoPolicy)
113616,"[""thumbnailphoto""]",(l=9736f74f-fd37-4b02-80e8-8120a72ad6c2)
113771,"[""""]","(&(objectCategory=user)(memberOf=CN=Domain Admins,CN=Users,DC=simulandlabs,DC=com))"


We see a few things that are interesting from the `attr_list`.  There is a mention of `thumbnailphoto`, where if we do a quick Google search for `adfs` and `thumbnailphoto`, we can see that this is a way from the actor to access the [Distributed Key Manager](https://threathunterplaybook.com/library/windows/adfs_dkm_keys.html) (DKM) Keys. 

Looking back at the steps that the actor needs to accomplish, we can see evidence that the actor has moved to Step 2.  There is an attribute value listed in the `search_filter` as well as searching for `users` in the `Domain Admins` group.  We may be able to hunt for more activity based on these items.


So far we have found activity for:
  - T1087.002 - Account Discovery: Domain Account
  - T1552.004 - Unsecured Credentials: Private Keys
  
  
Let's take a look at our win events for anything that might catch ADFS objects since the attribute value might be an object value.

In [10]:
thumbnail_object = FIND managed_entity RESPONDED win_events
                   WHERE uid LIKE '%9736f74f-fd37-4b02-80e8-8120a72ad6c2%'
DISP thumbnail_object

thumbnail_user = FIND user READ thumbnail_object
DISP thumbnail_user

thumbnail_event = FIND event RESPONDED BY thumbnail_object
DISP thumbnail_event ATTR time, event_id, type_uid, type_name, device.hostname SORT BY time ASC

uid 
 type 
 data 
 
 
 
 
 9736f74f-fd37-4b02-80e8-8120a72ad6c2 
 5cb41ed0-0e4c-11d0-a286-00aa003049e2 
 %%7684 \t\t{77b5b886-944a-11d1-aebd-0000f80367c1} \t\t\t{8d3bca50-1d7e-11d0-a081-00aa006c33ed} \t{5cb41ed0-0e4c-11d0-a286-00aa003049e2} 
 
 
 9736f74f-fd37-4b02-80e8-8120a72ad6c2 
 5cb41ed0-0e4c-11d0-a286-00aa003049e2 
 %%7684 \t\t{e48d0154-bcf8-11d1-8702-00c04fb96050} \t\t\t{bf9679e5-0de6-11d0-a285-00aa003049e2} \t\t\t{bf967a0e-0de6-11d0-a285-00aa003049e2} \t\t{77b5b886-944a-11d1-aebd-0000f80367c1} \t\t\t{8d3bca50-1d7e-11d0-a081-00aa006c33ed} \t{5cb41ed0-0e4c-11d0-a286-00aa003049e2} 
 
 
 9736f74f-fd37-4b02-80e8-8120a72ad6c2 
 5cb41ed0-0e4c-11d0-a286-00aa003049e2 
 %%7684 \t{5cb41ed0-0e4c-11d0-a286-00aa003049e2} \t\t{e48d0154-bcf8-11d1-8702-00c04fb96050} \t\t\t{bf9679e5-0de6-11d0-a285-00aa003049e2} \t\t\t{bf96793f-0de6-11d0-a285-00aa003049e2} \t\t\t{bf967a41-0de6-11d0-a285-00aa003049e2} \t\t\t{bf9679ef-0de6-11d0-a285-00aa003049e2} \t\t\t{bf9679f0-0de6-11d0-a285-00aa003049e2} \t\t\t{bf967a55-0de6-11d0-a285-00aa003049e2} \t\t\t{bf967950-0de6-11d0-a285-00aa003049e2} \t\t\t{f0f8ff8e-1191-11d0-a060-00aa006c33ed} \t\t\t{f0f8ff90-1191-11d0-a060-00aa006c33ed} \t\t\t{bf9679e4-0de6-11d0-a285-00aa003049e2} \t\t\t{f0f8ffa7-1191-11d0-a060-00aa006c33ed} \t\t\t{bf96794f-0de6-11d0-a285-00aa003049e2} \t\t\t{f0f8ff88-1191-11d0-a060-00aa006c33ed} \t\t\t{bf967a06-0de6-11d0-a285-00aa003049e2} \t\t\t{bf967954-0de6-11d0-a285-00aa003049e2} \t\t\t{bf967a1c-0de6-11d0-a285-00aa003049e2} \t\t\t{bf967a0e-0de6-11d0-a285-00aa003049e2} \t\t\t{bf9679e7-0de6-11d0-a285-00aa003049e2} \t\t\t{fe6136a0-2073-11d0-a9c2-00aa006c33ed} \t\t\t{6d05fb41-246b-11d0-a9c8-00aa006c33ed} \t\t\t{e0fa1e62-9b45-11d0-afdd-00c04fd930c9} \t\t\t{3e74f60e-3e73-11d1-a9c0-0000f80367c1} \t\t\t{0296c123-40da-11d1-a9c0-0000f80367c1} \t\t\t{28630ebc-41d5-11d1-a9c1-0000f80367c1} \t\t\t{26d97369-6070-11d1-a9c6-0000f80367c1} \t\t\t{800d94d7-b7a1-42a1-b14d-7cae1423d07f} \t\t\t{773e93af-d3b4-48d4-b3f9-06457602d3d0} \t\t\t{4b1cba4e-302f-4134-ac7c-f01f6c797843} \t\t\t{f217e4ec-0836-4b90-88af-2f5d4bbda2bc} \t\t\t{6cd53daf-003e-49e7-a702-6fa896e7a6ef} \t\t\t{5bd5208d-e5f4-46ae-a514-543bc9c47659} \t\t\t{e21a94e4-2d66-4ce5-b30d-0ef87a776ff0} \t\t\t{def449f1-fd3b-4045-98cf-d9658da788b5} \t\t\t{bf967961-0de6-11d0-a285-00aa003049e2} \t\t\t{bf9679b5-0de6-11d0-a285-00aa003049e2} \t\t{771727b1-31b8-4cdf-ae62-4fe39fadf89e} \t\t\t{bf967a78-0de6-11d0-a285-00aa003049e2} \t\t\t{bf967a77-0de6-11d0-a285-00aa003049e2} \t\t\t{bf967a3c-0de6-11d0-a285-00aa003049e2} \t\t\t{bf967a70-0de6-11d0-a285-00aa003049e2} \t\t\t{bf96798f-0de6-11d0-a285-00aa003049e2} \t\t\t{167757bc-47f3-11d1-a9c3-0000f80367c1} \t\t\t{16775848-47f3-11d1-a9c3-0000f80367c1} \t\t\t{bf967a1e-0de6-11d0-a285-00aa003049e2} \t\t\t{bf967a1d-0de6-11d0-a285-00aa003049e2} \t\t\t{bf9679f4-0de6-11d0-a285-00aa003049e2} \t\t\t{bf967a6f-0de6-11d0-a285-00aa003049e2} \t\t\t{bf967a73-0de6-11d0-a285-00aa003049e2} \t\t\t{bf96791a-0de6-11d0-a285-00aa003049e2} \t\t\t{bf967972-0de6-11d0-a285-00aa003049e2} \t\t\t{bf967a71-0de6-11d0-a285-00aa003049e2} \t\t\t{5fd424a1-1262-11d0-a060-00aa006c33ed} \t\t\t{a8df7394-c5ea-11d1-bbcb-0080c76670c0} \t\t\t{a8df7498-c5ea-11d1-bbcb-0080c76670c0} \t\t\t{16775820-47f3-11d1-a9c3-0000f80367c1} \t\t\t{a8df7407-c5ea-11d1-bbcb-0080c76670c0} \t\t\t{bf967a16-0de6-11d0-a285-00aa003049e2} \t\t\t{bf967962-0de6-11d0-a285-00aa003049e2} \t\t\t{bf967976-0de6-11d0-a285-00aa003049e2} \t\t\t{bf967a21-0de6-11d0-a285-00aa003049e2} \t\t\t{244b2970-5abd-11d0-afd2-00c04fd930c9} \t\t\t{66171887-8f3c-11d0-afda-00c04fd930c9} \t\t\t{3e10944d-c354-11d0-aff8-0000f80367c1} \t\t\t{26d9736e-6070-11d1-a9c6-0000f80367c1} \t\t\t{52458019-ca6a-11d0-afff-0000f80367c1} \t\t\t{e1aea404-cd5b-11d0-afff-0000f80367c1} \t\t\t{05308983-7688-11d1-aded-00c04fd8d5cd} \t\t\t{19405b9c-3cfa-11d1-a9c0-0000f80367c1} \t\t\t{19405b9e-3cfa-11d1-a9c0-0000f80367c1} \t\t\t{0296c122-40da-11d1-a9c0-0000f80367c1} \t\t\t{0296c124-40da-11d1-a9c0-0000f80367c1} \t\t\t{bf967951-0de6-11d0-a285-00aa00

domain,name,uid
SIMULANDLABS,wardog,S-1-5-21-2490359158-13971675-3780420524-500


time,event_id,type_uid,type_name,device.hostname
2021-08-02T13:11:39.77Z,4662,300400,An operation was performed on an object,dc01.simulandlabs.com
2021-08-02T13:11:53.63Z,4662,300400,An operation was performed on an object,dc01.simulandlabs.com
2021-08-02T13:13:11.003Z,4662,300400,An operation was performed on an object,dc01.simulandlabs.com
2021-08-02T13:13:11Z,4662,300400,An operation was performed on an object,dc01.simulandlabs.com


Looks like we see activity that is based off of the previous AD Object.  Windows Event ID `4662` shows any time an AD object was accessed and from what system that activity took place on.  If we look at the `entity.data` this looks like guids from additional objects and the range looks like this might be enumeration.  For how quickly these calls are being made, this looks very automated.  What tool might be able to do this type of activity on a DC?  

We have found activity for:
  - T1003 - OS Credential Dumping
  
  
We also have a rough time of the start of the attack `2021-08 13:11:00`

What other events happened around that time?

Let's see what was captured by MDE?

In [11]:
additional_events = mde_events
                    WHERE (device.hostname LIKE '%dc01%' OR device.hostname LIKE '%adfs01%')
                      AND type_name != "LdapSearch" # exclude the LdapSearch we already explored

DISP additional_events

time,device.hostname,file.endpoint.hostname,process.endpoint.hostname,actor.process.endpoint.hostname,actor.process.parent_process.endpoint.hostname,actor.user.endpoint.hostname,user.endpoint.hostname,actor.user.uid,actor.user.name,user.name,actor.user.domain,user.domain,src_endpoint.ip,src_endpoint.port,dst_endpoint.hostname,dst_endpoint.ip,dst_endpoint.port,user.uid,query_info.uid,entity.uid,type_uid,type_name
2021-08-02T13:13:02.057Z,adfs01.simulandlabs.com,adfs01.simulandlabs.com,adfs01.simulandlabs.com,adfs01.simulandlabs.com,adfs01.simulandlabs.com,adfs01.simulandlabs.com,adfs01.simulandlabs.com,S-1-5-21-2490359158-13971675-3780420524-1105,pgustavo,pgustavo,simulandlabs.com,simulandlabs.com,192.168.2.5,53163,dc01.simulandlabs.com,192.168.2.4,49668,aead923d-498b-4f64-a66c-2af91447a8b6,3ab932f1-c0d9-4c09-bb4d-dfc0852bbdf7_106830890_1627909982057_20940,3ab932f1-c0d9-4c09-bb4d-dfc0852bbdf7_106830890_1627909982057_20940,600599,Directory Services replication
2021-08-02T13:08:18.374Z,adfs01.simulandlabs.com,adfs01.simulandlabs.com,adfs01.simulandlabs.com,adfs01.simulandlabs.com,adfs01.simulandlabs.com,adfs01.simulandlabs.com,adfs01.simulandlabs.com,S-1-5-21-2490359158-13971675-3780420524-1105,pgustavo,pgustavo,simulandlabs.com,simulandlabs.com,192.168.2.5,53115,dc01.simulandlabs.com,192.168.2.4,49668,aead923d-498b-4f64-a66c-2af91447a8b6,85a25128-abab-4d26-8906-5fe01c1afe17_106830890_1627909698374_20940,85a25128-abab-4d26-8906-5fe01c1afe17_106830890_1627909698374_20940,600599,Directory Services replication


During the ldapsearch we see that there was a user with details that were changed?

Let's zoom into the strange part:

In [12]:
additional_events_actor = FIND user ORIGINATED additional_events
DISP additional_events_actor

additional_events_user = FIND user RESPONDED additional_events
DISP additional_events_user

endpoint.hostname,uid,name,domain
adfs01.simulandlabs.com,S-1-5-21-2490359158-13971675-3780420524-1105,pgustavo,simulandlabs.com


endpoint.hostname,name,domain,uid
adfs01.simulandlabs.com,pgustavo,simulandlabs.com,aead923d-498b-4f64-a66c-2af91447a8b6


Let's follow that trail around to the other datasources.

In [13]:
aad_related_events = GET event FROM sqlalchemy://GoldenSAML-AADAuditEvents
                     WHERE actor.user.uid = additional_events_actor.uid
                        OR actor.user.uid = additional_events_user.uid

DISP aad_related_events

time,device.uid,actor.user.endpoint.uid,http_request.user_agent,entity.uid,entity.type,entity.name,entity.data,actor.user.uid,actor.user.name,privileges,type_uid,type_name,status_id
2021-08-02T13:29:25.983Z,00000000-0000-0000-0000-000000000000,00000000-0000-0000-0000-000000000000,Mozilla/5.0 (Windows NT; Windows NT 10.0; en-US) WindowsPowerShell/5.1.17763.1971,11b49e19-2326-4be6-93cb-7f37439bbd81,Application,SimuLandApp,"[{""id"": ""11b49e19-2326-4be6-93cb-7f37439bbd81"", ""displayName"": ""SimuLandApp"", ""type"": ""Application"", ""modifiedProperties"": [{""displayName"": ""KeyDescription"", ""oldValue"": ""[]"", ""newValue"": ""[\""[KeyIdentifier=59eaeebc-8a6b-44a6-9f24-7d55e64420c9,KeyType=Password,KeyUsage=Verify,DisplayName=SimuLandCreds]\""]""}, {""displayName"": ""Included Updated Properties"", ""oldValue"": null, ""newValue"": ""\""KeyDescription\""""}], ""administrativeUnits"": []}]",aead923d-498b-4f64-a66c-2af91447a8b6,pgustavo@simulandlabs.com,None,300403,Update application – Certificates and secrets management,1
2021-08-02T13:25:12.246Z,00000000-0000-0000-0000-000000000000,00000000-0000-0000-0000-000000000000,Mozilla/5.0 (Windows NT; Windows NT 10.0; en-US) WindowsPowerShell/5.1.17763.1971,11b49e19-2326-4be6-93cb-7f37439bbd81,Application,SimuLandApp,"[{""id"": ""11b49e19-2326-4be6-93cb-7f37439bbd81"", ""displayName"": ""SimuLandApp"", ""type"": ""Application"", ""modifiedProperties"": [{""displayName"": ""RequiredResourceAccess"", ""oldValue"": ""[{\""ResourceAppId\"":\""00000003-0000-0000-c000-000000000000\"",\""RequiredAppPermissions\"":[{\""EntitlementId\"":\""e1fe6dd8-ba31-4d61-89e7-88639da4683d\"",\""DirectAccessGrant\"":false,\""ImpersonationAccessGrants\"":[20]}],\""EncodingVersion\"":1}]"", ""newValue"": ""[{\""ResourceAppId\"":\""00000003-0000-0000-c000-000000000000\"",\""RequiredAppPermissions\"":[{\""EntitlementId\"":\""e1fe6dd8-ba31-4d61-89e7-88639da4683d\"",\""DirectAccessGrant\"":false,\""ImpersonationAccessGrants\"":[20]},{\""EntitlementId\"":\""024d486e-b451-40bb-833d-3e66d98c5c73\"",\""DirectAccessGrant\"":false,\""ImpersonationAccessGrants\"":[20]}],\""EncodingVersion\"":1}]""}, {""displayName"": ""Included Updated Properties"", ""oldValue"": null, ""newValue"": ""\""RequiredResourceAccess\""""}], ""administrativeUnits"": []}]",aead923d-498b-4f64-a66c-2af91447a8b6,pgustavo@simulandlabs.com,None,300403,Update application,1
2021-08-02T13:27:20.017Z,00000000-0000-0000-0000-000000000000,00000000-0000-0000-0000-000000000000,Mozilla/5.0 (Windows NT; Windows NT 10.0; en-US) WindowsPowerShell/5.1.17763.1971,None,ServicePrincipal,None,"[{""id"": ""401dd906-ea4f-4d41-b762-7e936d222368"", ""displayName"": ""Microsoft Graph"", ""type"": ""ServicePrincipal"", ""modifiedProperties"": [{""displayName"": ""DelegatedPermissionGrant.Scope"", ""oldValue"": ""\""User.Read\"""", ""newValue"": ""\""User.Read Mail.ReadWrite\""""}, {""displayName"": ""DelegatedPermissionGrant.ConsentType"", ""oldValue"": ""\""AllPrincipals\"""", ""newValue"": ""\""AllPrincipals\""""}, {""displayName"": ""ServicePrincipal.ObjectID"", ""oldValue"": null, ""newValue"": ""\""0d2f5969-011b-460d-ac74-3291d227d49f\""""}, {""displayName"": ""ServicePrincipal.DisplayName"", ""oldValue"": null, ""newValue"": null}, {""displayName"": ""ServicePrincipal.AppId"", ""oldValue"": null, ""newValue"": null}, {""displayName"": ""ServicePrincipal.Name"", ""oldValue"": null, ""newValue"": null}, {""displayName"": ""TargetId.ServicePrincipalNames"", ""oldValue"": null, ""newValue"": ""\""00000003-0000-0000-c000-000000000000/ags.windows.net;00000003-0000-0000-c000-000000000000;https://canary.graph.microsoft.com;https://graph.microsoft.com;https://ags.windows.net;https://graph.microsoft.us;https://graph.microsoft.com/;https://dod-graph.microsoft.us;https://canary.graph.microsoft.com/;https://graph.microsoft.us/;https://dod-graph.microsoft.us/\""""}], ""administrativeUnits"": []}, {""id"": ""0d2f5969-011b-460d-ac74-3291d227d49f"", ""displayName"": null, ""type"

We see that user doing interesting Application actions with a user_agent showing PowerShell.  Could this be related to the other malicious activity observed using PowerShell?  There are also privileges added and referencing Microsoft Graph.  Why would a user access their own mailbox with an API? Let's see if those privileges were successfully used.

We have found activity for:
  - T1098.002: Additional Email Delegate Permissions
  - T1134: Access Token Manipulation

In [14]:
priv_mde = mde_events WHERE privileges LIKE "%Mail.ReadWrite%"

DISP priv_mde SORT BY time ASC

time,user.uid,query_info.uid,entity.uid,entity.data,privileges,type_uid,type_name
2021-08-02T13:27:20Z,aead923d-498b-4f64-a66c-2af91447a8b6,106830890_11161_e417c1fd-ae0b-4efa-891b-bb6b0ea7d35e,106830890_11161_e417c1fd-ae0b-4efa-891b-bb6b0ea7d35e,"[{""ApplicationInstance"": 0, ""ApplicationId"": 11161, ""Type"": ""User"", ""Role"": ""Actor"", ""Name"": ""Gustavo Pedro"", ""Id"": ""aead923d-498b-4f64-a66c-2af91447a8b6""}]",User.Read Mail.ReadWrite,300501,Add delegated permission grant.


Finally, Let's see if the application was access

In [15]:
aad_users = FIND user ORIGINATED aad_related_events

DISP aad_users

uid_office = GET event
             FROM sqlalchemy://GoldenSAML-OfficeActivityEvents
             WHERE actor.user.name = aad_users.name

DISP uid_office ATTR time, src_endpoint.ip, type_name, entity.type, entity.data, actor.user.uid, actor.user.name

endpoint.uid,uid,name
00000000-0000-0000-0000-000000000000,aead923d-498b-4f64-a66c-2af91447a8b6,pgustavo@simulandlabs.com


time,src_endpoint.ip,type_name,entity.type,entity.data,actor.user.uid,actor.user.name
2021-08-02T13:32:07Z,1.2.3.4,MailItemsAccessed,OfficeActivity,"[{""FolderItems"": [{""InternetMessageId"": ""<00000000-0000-0000-0000-000000000000@az.centralus.production.microsoft.com>""}, {""InternetMessageId"": ""<AB1CD23EF4567DAC24353B8AD047DDEB0B1EB9@AB1CD23EF4567.abcdef12.prod.outlook.com>""}, {""InternetMessageId"": ""<00000000-0000-0000-0000-000000000000@az.southcentralus.production.microsoft.com>""}, {""InternetMessageId"": ""<00000000-0000-0000-0000-000000000000@az.eastus.production.microsoft.com>""}, {""InternetMessageId"": ""<00000000-0000-0000-0000-000000000000@az.northcentralus.production.microsoft.com>""}, {""InternetMessageId"": ""<00000000-0000-0000-0000-000000000000@az.northcentralus.production.microsoft.com>""}, {""InternetMessageId"": ""<00000000-0000-0000-0000-000000000000@az.westus2.production.microsoft.com>""}], ""Id"": ""LgAAAAAM7KyTTmWeRac2KXBEz/7aAQARGHK+grzLTpRJraC1QR6kAAAAAAEMAAAB"", ""Path"": ""\\Inbox""}]",S-1-5-21-1825954961-3338807533-2873504967-26087451,pgustavo@simulandlabs.com
